In [1]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
import time
from tkinter import TclError
from scipy.fftpack import fft
# Necesitaremos fft: La transformada rápida de fourier

In [3]:
p = pyaudio.PyAudio()

In [4]:
p.get_default_input_device_info()

{'index': 1,
 'structVersion': 2,
 'name': 'Mic in at front panel (black) (',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [5]:
p.get_device_info_by_index(1)

{'index': 1,
 'structVersion': 2,
 'name': 'Mic in at front panel (black) (',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [6]:
# Se utiliza para que las figuras de matplotlib se abran en otra ventana aparte
%matplotlib tk

# constantes
CHUNK = 1024 * 64                 # número de objetos (de tipo FORMAT) almacenados a la vez
FORMAT = pyaudio.paInt16     # Tipo de dato de cada observación
CHANNELS = 1                 # Un canal para el micrófono
RATE = 44100                 # muestras por segundo, i.e 44.1 kHz, estándar de la industria 
                             # para CDs, mp3. También 48 kHz es común para DVDs y Blu-rays
# input_device_index=13

In [7]:
notes=["Do","Do#","Re","Re#","Mi","Fa","Fa#","Sol","Sol#","La","La#","Si",
       "2Do","2Do#","2Re","2Re#","2Mi","2Fa","2Fa#","2Sol","2Sol#","2La","2La#","2Si",
      "3Do","3Do#","3Re","3Re#","3Mi","3Fa","3Fa#","3Sol","3Sol#","3La","3La#","3Si",
      "4Do","4Do#","4Re","4Re#","4Mi","4Fa","4Fa#","4Sol","4Sol#","4La","4La#","4Si"]
# Esta lista contiene las notas musicales y sus octavas, entonces el 2, 3 y 4 antes de la nota denotarán que es 2, 3, o 4 
# octavas por encima de la nota, tomé la nota Do que se encuentra 4 semitonos antes de la nota mi que genera la 
# sexta cuerda en la afinación estadar de guitarra.

In [8]:
fo=65.48
r=1.059463
data_frec_min=[fo-4]
data_frec_max=[fo+4]
for d in range(48):
    data_frec_min.append(fo*r**(d+1)-4)
    data_frec_max.append(fo*r**(d+1)+4)
dfmin=np.array(data_frec_min)
dfmax=np.array(data_frec_max)

In [9]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=False,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-2**15, 2**15-1)
#ax1.set_ylim(0, 255)
#ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)
ax2.set_ylim(0, 25)
ax2.set_xlabel('Frecuencias')

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    #data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    #data_np = np.array(data_int, dtype='b')[::2] + 128
    data_np = struct.unpack(str(CHUNK) + 'h', data)
    
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_np)
    auxiliar=np.where(yf==np.amax(yf))
    indice=auxiliar[0][0]
    if(xf[indice]<=20000):
        print(xf[indice])
        contador=0
        for x, y in zip(dfmin, dfmax):
            if(x<=xf[indice]<=y):
                nota=notes[contador]
                print(nota)
                break
            contador+=1
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        # Cerramos el canal de audio
        p.close(stream)
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started
174.95994506752118
2Fa
174.28702220187688
2Fa
275.898374914168
3Do#
337.13435568780045
337.13435568780045
327.04051270313573
3Mi
329.0592813000687
3Mi
390.2952620737011
3Sol
438.0727855344473
3La
440.09155413138024
3La
549.1050583657589
35.66491187914855
657.445639734493
4Mi
413.17463950560773
3Sol#
2331.0048065918977
318.2925154497597
738.1963836118106
4Fa#
781.2634470130465
4Sol
826.3492790112155
1958.878461890593
232.83131151293205
2La#
656.0997940032045
4Mi
658.1185626001374
4Mi
492.57953765163654
3Si
553.1425955596246
4Do#
553.815518425269
4Do#
446.82078278782336
492.57953765163654
3Si
493.9253833829252
3Si
492.57953765163654
3Si
492.57953765163654
3Si
493.9253833829252
3Si
390.2952620737011
3Sol
439.4186312657359
3La
391.64110780498976
3Sol
328.3863584344244
3Mi
405.7724879835203
stream stopped
average frame rate = 1 FPS


In [ ]:
#Los datos mostrados anteriormente sin correr la celda, son los que generé al tocar notas de la primera cuerda
#en guitarra acústica. Aveces le atina a la nota otras no.